In [1]:
# import necessary libraries
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# define the paths to the train and test folders on your desktop
train_path = 'C:/Users/chand/Desktop/New folder/DeepLearning/Dataset/train'
test_path = 'C:/Users/chand/Desktop/New folder/DeepLearning/Dataset/test'

# define image and batch size
img_height, img_width = 28, 28
batch_size = 64

# create ImageDataGenerator objects for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(
    rescale=1.0/255.0
)

# create flow_from_directory generators for train and test datasets
train_generator = train_datagen.flow_from_directory(
    directory=train_path,
    target_size=(img_height, img_width),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

test_generator = test_datagen.flow_from_directory(
    directory=test_path,
    target_size=(img_height, img_width),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False,
    seed=42
)

# define the model architecture
model = keras.Sequential(
    [
        keras.Input(shape=(img_height, img_width, 1)),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dense(train_generator.num_classes, activation="softmax"),
    ]
)

# compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# train the model
# MODEL.summary

history = model.fit(train_generator, epochs=5, validation_data=test_generator)

# evaluate the model on test set
test_loss, test_acc = model.evaluate(test_generator, verbose=2)
print("Test accuracy:", test_acc)


Found 17000 images belonging to 10 classes.
Found 2999 images belonging to 10 classes.
Epoch 1/5
266/266 [==============================] - 33s 119ms/step - loss: 1.1123 - accuracy: 0.6334 - val_loss: 0.3322 - val_accuracy: 0.8813
Epoch 2/5
266/266 [==============================] - 32s 120ms/step - loss: 0.4920 - accuracy: 0.8441 - val_loss: 0.1721 - val_accuracy: 0.9453
Epoch 3/5
266/266 [==============================] - 32s 121ms/step - loss: 0.3583 - accuracy: 0.8872 - val_loss: 0.1049 - val_accuracy: 0.9697
Epoch 4/5
266/266 [==============================] - 30s 111ms/step - loss: 0.2960 - accuracy: 0.9084 - val_loss: 0.0856 - val_accuracy: 0.9740
Epoch 5/5
266/266 [==============================] - 32s 119ms/step - loss: 0.2581 - accuracy: 0.9192 - val_loss: 0.0710 - val_accuracy: 0.9787
47/47 - 2s - loss: 0.0710 - accuracy: 0.9787 - 2s/epoch - 48ms/step
Test accuracy: 0.9786595702171326


In [2]:
optimizers = [tf.keras.optimizers.SGD(),tf.keras.optimizers.Adam(),tf.keras.optimizers.Adadelta(),tf.keras.optimizers.RMSprop(),tf.keras.optimizers.Adagrad(),tf.keras.optimizers.Adamax(),tf.keras.optimizers.Nadam()]

# loop through each optimizer and train the model
for optimizer in optimizers:
    
    # reset the model weights for each optimizer
    model = keras.Sequential(
        [
            keras.Input(shape=(img_height, img_width, 1)),
            layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Flatten(),
            layers.Dense(128, activation="relu"),
            layers.Dense(train_generator.num_classes, activation="softmax"),
        ]
    )
    
    # compile the model with the current optimizer
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
    
    # train the model
    history = model.fit(train_generator, epochs=10, validation_data=test_generator, verbose=0)
    
    # evaluate the model on test set
    test_loss, test_acc = model.evaluate(test_generator, verbose=0)
    test_predictions = model.predict(test_generator)
    test_pred_labels = np.argmax(test_predictions, axis=1)
    test_true_labels = test_generator.classes
    
    # calculate precision, recall, and F-score
    precision = tf.keras.metrics.Precision()(test_true_labels, test_pred_labels)
    recall = tf.keras.metrics.Recall()(test_true_labels, test_pred_labels)
    f1_score = 2 * (precision * recall) / (precision + recall)
    
    # print the evaluation metrics
    print("Optimizer:", optimizer.__class__.__name__)
    print("Test loss:", test_loss)
    print("Test accuracy:", test_acc)
    print("Precision:", precision.numpy())
    print("Recall:", recall.numpy())
    print("F-score:", f1_score.numpy())
    print("\n")


47/47 [==============================] - 2s 46ms/step
Optimizer: SGD
Test loss: 0.3450772762298584
Test accuracy: 0.8989663124084473
Precision: 0.9867938
Recall: 0.9962963
F-score: 0.99152225


47/47 [==============================] - 3s 53ms/step
Optimizer: Adam
Test loss: 0.05208950489759445
Test accuracy: 0.9833277463912964
Precision: 1.0
Recall: 1.0
F-score: 1.0


47/47 [==============================] - 2s 49ms/step
Optimizer: Adadelta
Test loss: 1.9836329221725464
Test accuracy: 0.5055018067359924
Precision: 0.9133378
Recall: 0.99925923
F-score: 0.9543686


47/47 [==============================] - 2s 44ms/step
Optimizer: RMSprop
Test loss: 0.04784171283245087
Test accuracy: 0.9869956374168396
Precision: 0.9988901
Recall: 1.0
F-score: 0.9994447


47/47 [==============================] - 2s 45ms/step
Optimizer: Adagrad
Test loss: 0.8353080749511719
Test accuracy: 0.811603844165802
Precision: 0.99924815
Recall: 0.98444444
F-score: 0.991791


47/47 [==============================] - 

In [3]:
from tensorflow.keras import regularizers

# define a list of regularizers to try
regularizers = [regularizers.l1(0.01), regularizers.l2(0.01), regularizers.l1_l2(l1=0.01, l2=0.01)]

# loop through each regularizer and train the model
for regularizer in regularizers:
    
    # reset the model weights for each regularizer
    model = keras.Sequential(
        [
            keras.Input(shape=(img_height, img_width, 1)),
            layers.Conv2D(32, kernel_size=(3, 3), activation="relu", kernel_regularizer=regularizer),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Flatten(),
            layers.Dense(128, activation="relu", kernel_regularizer=regularizer),
            layers.Dense(train_generator.num_classes, activation="softmax"),
        ]
    )
    
    # compile the model with default optimizer
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    
    # train the model
    history = model.fit(train_generator, epochs=10, validation_data=test_generator, verbose=0)
    
    # evaluate the model on test set
    test_loss, test_acc = model.evaluate(test_generator, verbose=0)
    test_predictions = model.predict(test_generator)
    test_pred_labels = np.argmax(test_predictions, axis=1)
    test_true_labels = test_generator.classes
    
    # calculate precision, recall, and F-score
    precision = tf.keras.metrics.Precision()(test_true_labels, test_pred_labels)
    recall = tf.keras.metrics.Recall()(test_true_labels, test_pred_labels)
    f1_score = 2 * (precision * recall) / (precision + recall)
    
    # print the evaluation metrics
    print("Regularizer:", regularizer.__class__.__name__)
    print("Test loss:", test_loss)
    print("Test accuracy:", test_acc)
    print("Precision:", precision.numpy())
    print("Recall:", recall.numpy())
    print("F-score:", f1_score.numpy())
    print("\n")

47/47 [==============================] - 2s 50ms/step
Regularizer: L1
Test loss: 3.0770576000213623
Test accuracy: 0.10003334283828735
Precision: 0.9003001
Recall: 1.0
F-score: 0.9475347


47/47 [==============================] - 3s 57ms/step
Regularizer: L2
Test loss: 0.6127364635467529
Test accuracy: 0.9089696407318115
Precision: 0.985031
Recall: 0.99925923
F-score: 0.9920941


47/47 [==============================] - 2s 47ms/step
Regularizer: L1L2
Test loss: 3.0720601081848145
Test accuracy: 0.10003334283828735
Precision: 0.9003001
Recall: 1.0
F-score: 0.9475347




In [7]:
train_data = 'C:/Users/chand/Desktop/New folder/DeepLearning/Dataset/train'
test_data = 'C:/Users/chand/Desktop/New folder/DeepLearning/Dataset/test'
train_labels='C:/Users/chand/Desktop/New folder/DeepLearning/Dataset/test'
test_labels = 'C:/Users/chand/Desktop/New folder/DeepLearning/Dataset/test'


In [17]:
# import necessary libraries
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# define the paths to the train and test folders on your desktop
train_path = 'C:/Users/chand/Desktop/New folder/DeepLearning/Dataset/train'
test_path = 'C:/Users/chand/Desktop/New folder/DeepLearning/Dataset/test'

# define image and batch size
img_height, img_width = 28, 28
batch_size = 64

# create ImageDataGenerator objects for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(
    rescale=1.0/255.0
)

# create flow_from_directory generators for train and test datasets
train_generator = train_datagen.flow_from_directory(
    directory=train_path,
    target_size=(img_height, img_width),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

test_generator = test_datagen.flow_from_directory(
    directory=test_path,
    target_size=(img_height, img_width),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False,
    seed=42
)

# define the CNN architecture
cnn = keras.Sequential(
    [
        keras.Input(shape=(img_height, img_width, 1)),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
    ]
)

# define the LSTM architecture
rnn = keras.Sequential(
    [
        layers.Reshape((1, 128)),
        layers.LSTM(64),
        layers.Dense(train_generator.num_classes, activation="softmax"),
    ]
)

# combine the CNN and RNN architectures
model = keras.Model(inputs=cnn.input, outputs=rnn(cnn.output))

# compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# train the model
history = model.fit(train_generator, epochs=5, validation_data=test_generator)

# evaluate the model on test set
test_loss, test_acc = model.evaluate(test_generator, verbose=2)
print("Test accuracy:", test_acc)


Found 17000 images belonging to 10 classes.
Found 2999 images belonging to 10 classes.
Epoch 1/5
266/266 [==============================] - 37s 121ms/step - loss: 1.1990 - accuracy: 0.6060 - val_loss: 0.3453 - val_accuracy: 0.9080
Epoch 2/5
266/266 [==============================] - 31s 115ms/step - loss: 0.5495 - accuracy: 0.8254 - val_loss: 0.1892 - val_accuracy: 0.9503
Epoch 3/5
266/266 [==============================] - 31s 115ms/step - loss: 0.3683 - accuracy: 0.8861 - val_loss: 0.1484 - val_accuracy: 0.9563
Epoch 4/5
266/266 [==============================] - 33s 123ms/step - loss: 0.2929 - accuracy: 0.9070 - val_loss: 0.1086 - val_accuracy: 0.9697
Epoch 5/5
266/266 [==============================] - 34s 127ms/step - loss: 0.2377 - accuracy: 0.9281 - val_loss: 0.0811 - val_accuracy: 0.9783
47/47 - 2s - loss: 0.0811 - accuracy: 0.9783 - 2s/epoch - 49ms/step
Test accuracy: 0.9783260822296143


In [7]:
# import necessary libraries
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# define the paths to the train and test folders on your desktop
train_path = 'C:/Users/chand/Desktop/New folder/DeepLearning/Dataset/train'
test_path = 'C:/Users/chand/Desktop/New folder/DeepLearning/Dataset/test'

# define image and batch size
img_height, img_width = 28, 28
batch_size = 64

# create ImageDataGenerator objects for data augmentation
train_datagen = ImageDataGenerator(
rescale=1.0/255.0,
rotation_range=20,
zoom_range=0.2,
shear_range=0.2,
width_shift_range=0.2,
height_shift_range=0.2,
horizontal_flip=False,
vertical_flip=False,
fill_mode='nearest'
)

test_datagen = ImageDataGenerator(
rescale=1.0/255.0
)

# create flow_from_directory generators for train and test datasets
train_generator = train_datagen.flow_from_directory(
directory=train_path,
target_size=(img_height, img_width),
color_mode="grayscale",
batch_size=batch_size,
class_mode="categorical",
shuffle=True,
seed=42
)

test_generator = test_datagen.flow_from_directory(
directory=test_path,
target_size=(img_height, img_width),
color_mode="grayscale",
batch_size=batch_size,
class_mode="categorical",
shuffle=False,
seed=42
)

# define the CNN architecture
cnn = keras.Sequential(
[
keras.Input(shape=(img_height, img_width, 1)),
layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
layers.MaxPooling2D(pool_size=(2, 2)),
layers.Flatten(),
layers.Dense(128, activation="relu"),
]
)

# compile the CNN model
cnn.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# train the CNN model
history_cnn = cnn.fit(train_generator, epochs=5, validation_data=test_generator)

# evaluate the CNN model on test set
test_loss_cnn, test_acc_cnn = cnn.evaluate(test_generator, verbose=2)
print("CNN Test accuracy:", test_acc_cnn)

# define the RNN architecture
rnn = keras.Sequential(
[
layers.LSTM(64, input_shape=(1, 128)),
layers.Dense(train_generator.num_classes, activation="softmax"),
]
)

# compile the RNN model
rnn.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# prepare data for RNN training
def extract_features(model, data_generator):
    features = model.predict(data_generator)
    features = features.reshape((features.shape[0], 1, features.shape[1]))
    return features

train_features = extract_features(cnn, train_generator)
test_features = extract_features(cnn, test_generator)

# train the RNN model
history_rnn = rnn.fit(train_features, epochs=5, validation_data=(test_features, test_generator.labels))

# evaluate the RNN model on test set
test_loss_rnn, test_acc_rnn = rnn.evaluate(test_features, test_generator.labels, verbose=2)
print("RNN Test accuracy:", test_acc_rnn)

# compare the performance of CNN and RNN models
import matplotlib.pyplot as plt

# plot training and validation accuracy
plt.plot(history_cnn.history['accuracy'], label='CNN training accuracy', color='blue')
plt.plot(history_cnn.history)

Found 17000 images belonging to 10 classes.
Found 2999 images belonging to 10 classes.
Epoch 1/5


InvalidArgumentError: Graph execution error:

Detected at node 'gradient_tape/categorical_crossentropy/mul/BroadcastGradientArgs' defined at (most recent call last):
    File "C:\Users\chand\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\chand\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\chand\anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\chand\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\chand\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "C:\Users\chand\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\chand\anaconda3\lib\asyncio\base_events.py", line 601

In [9]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths to the train and test folders
train_path = 'C:/Users/chand/Desktop/New folder/DeepLearning/Dataset/train'
test_path = 'C:/Users/chand/Desktop/New folder/DeepLearning/Dataset/test'

# Define image and batch size
img_height, img_width = 28, 28
batch_size = 64

# Create ImageDataGenerator objects for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(
    rescale=1.0/255.0
)

# Create flow_from_directory generators for train and test datasets
train_generator = train_datagen.flow_from_directory(
    directory=train_path,
    target_size=(img_height, img_width),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

test_generator = test_datagen.flow_from_directory(
    directory=test_path,
    target_size=(img_height, img_width),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False,
    seed=42
)

# Define the CNN architecture
cnn = keras.Sequential(
    [
        keras.Input(shape=(img_height, img_width, 1)),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dense(train_generator.num_classes, activation="softmax"),
    ]
)

# Define the LSTM architecture
rnn = keras.Sequential(
    [
        layers.Reshape((img_height, img_width)),
        layers.Bidirectional(layers.LSTM(64)),
        layers.Dense(train_generator.num_classes, activation="softmax"),
    ]
)

# Compile the models
cnn.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
rnn.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train the CNN
cnn_history = cnn.fit(train_generator, epochs=5, validation_data=test_generator)

# Train the RNN
rnn_history = rnn.fit(train_generator, epochs=5, validation_data=test_generator)

# Evaluate the models on test set
cnn_test_loss, cnn_test_acc = cnn.evaluate(test_generator, verbose=2)
rnn_test_loss, rnn_test_acc = rnn.evaluate(test_generator, verbose=2)

print("CNN Test accuracy:", cnn_test_acc)
print("RNN Test accuracy:", rnn_test_acc)


Found 17000 images belonging to 10 classes.
Found 2999 images belonging to 10 classes.
Epoch 1/5
246/266 [==========================>...] - ETA: 2s - loss: 1.1188 - accuracy: 0.6293

KeyboardInterrupt: 